# Data Validation

Compare data updated using the classic and modern pipeline. The current scope of validation is to ensure completeness and integrity. 

In [36]:
import pandas as pd
import numpy as np
import dataframe_image as dfi
import pyodbc
from sqlalchemy.engine import URL
import sqlalchemy as sa

In [38]:
host = #Provide host
user = #Provide username
password = #Provide password
database = 'rdb_modern'

In [27]:
#SQLACLHEMY
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER="+host+";DATABASE="+database+";UID="+user+";PWD="+password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
from sqlalchemy import create_engine
engine = create_engine(connection_url)

In [28]:
with engine.begin() as conn:
    sql_rdb_mod = pd.read_sql_query(sa.text("Select * from rdb_modern.dbo.HEPATITIS_DATAMART hd where hd.PATIENT_UID  = '101831376'"), conn)
    sql_rdb = pd.read_sql_query(sa.text("Select * from rdb.dbo.HEPATITIS_DATAMART hd where hd.PATIENT_UID  = '101831376'"), conn)


In [29]:
sql_rdb

,INIT_NND_NOT_DT,CASE_RPT_MMWR_WEEK,CASE_RPT_MMWR_YEAR,HEP_D_INFECTION_IND,HEP_MEDS_RECVD_IND,HEP_C_TOTAL_ANTIBODY,DIAGNOSIS_DT,DIE_FRM_THIS_ILLNESS_IND,DISEASE_IMPORTED_IND,EARLIEST_RPT_TO_CNTY,...,RPT_SRC_CITY,RPT_SRC_COUNTY,RPT_SRC_COUNTY_CD,PHYSICIAN_UID,PATIENT_UID,CASE_UID,INVESTIGATOR_UID,REPORTING_SOURCE_UID,REFRESH_DATETIME,PAT_BIRTH_COUNTRY
0,None,1.0,2021.0,None,Unknown,None,None,None,Unknown,None,...,None,None,None,26166015.0,101831376.0,101831381.0,234214498.0,34875455.0,2024-01-08 13:33:53.710,None


# Highlight Dataframe

In [30]:
df_combine = pd.concat([sql_rdb, sql_rdb_mod], axis='columns', keys=['Q1', 'Q2'])

In [31]:
df_axis= df_combine.swaplevel(axis='columns')[sql_rdb.columns[1:]]

In [32]:
df_axis

CASE_RPT_MMWR_WEEK      CASE_RPT_MMWR_YEAR         HEP_D_INFECTION_IND   
                  Q1   Q2                 Q1      Q2                  Q1   
0                1.0  1.0             2021.0  2021.0                None  \

        HEP_MEDS_RECVD_IND       HEP_C_TOTAL_ANTIBODY        ...     CASE_UID   
     Q2                 Q1    Q2                   Q1    Q2  ...           Q1   
0  None            Unknown  None                 None  None  ...  101831381.0  \

               INVESTIGATOR_UID              REPORTING_SOURCE_UID               
            Q2               Q1           Q2                   Q1          Q2   
0  101831381.0      234214498.0  234214498.0           34875455.0  34875455.0  \

         REFRESH_DATETIME                         PAT_BIRTH_COUNTRY        
                       Q1                      Q2                Q1    Q2  
0 2024-01-08 13:33:53.710 2024-01-15 17:22:37.130              None  None  

[1 rows x 412 columns]

In [33]:
df_axis.fillna('NAN', inplace=True)

In [34]:
def highlight_id(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Q1', axis='columns', level=-1)
    df = pd.DataFrame(np.where((data.ne(other, level=0)), attr, ''),
                        index=data.index, columns=data.columns)
    df = df[df.apply(lambda row: row.astype(str).str.contains('background-color: yellow', case=False).any(), axis=1)]
    id_list = df.index.get_level_values(0).to_list()

    return id_list

id_l = highlight_id(df_axis)
df = df_axis[df_axis.index.get_level_values(0).isin(id_l)]

df

CASE_RPT_MMWR_WEEK      CASE_RPT_MMWR_YEAR         HEP_D_INFECTION_IND        
                  Q1   Q2                 Q1      Q2                  Q1   Q2   
0                1.0  1.0             2021.0  2021.0                 NAN  NAN  \

  HEP_MEDS_RECVD_IND      HEP_C_TOTAL_ANTIBODY       ...     CASE_UID   
                  Q1   Q2                   Q1   Q2  ...           Q1   
0            Unknown  NAN                  NAN  NAN  ...  101831381.0  \

               INVESTIGATOR_UID              REPORTING_SOURCE_UID               
            Q2               Q1           Q2                   Q1          Q2   
0  101831381.0      234214498.0  234214498.0           34875455.0  34875455.0  \

         REFRESH_DATETIME                         PAT_BIRTH_COUNTRY       
                       Q1                      Q2                Q1   Q2  
0 2024-01-08 13:33:53.710 2024-01-15 17:22:37.130               NAN  NAN  

[1 rows x 412 columns]

In [35]:
def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Q1', axis='columns', level=-1)
    df = pd.DataFrame(np.where((data.ne(other, level=0)), attr, ''),
                        index=data.index, columns=data.columns)
    df = df[df.apply(lambda row: row.astype(str).str.contains('background-color: yellow', case=False).any(), axis=1)]

    return df

df.style.apply(highlight_diff, axis=None)

# Column View

View the only the differences side by side. 
Identify differences between two pandas DataFrames

In [24]:
def diff_func(df1, df2):
    assert (df1.columns == df2.columns).all(), \
        "DataFrame column names are different"
    if any(df1.dtypes != df2.dtypes):
        "Data Types are different, trying to convert"
        df2 = df2.astype(df1.dtypes)
    if df1.equals(df2):
        return None
    else:
        diff_mask = (df1 != df2) & ~(df1.isnull() & df2.isnull())
        ne_stacked = diff_mask.stack()
        changed = ne_stacked[ne_stacked]
        changed.index.names = ['ID', 'Column']
        difference_locations = np.where(diff_mask)
        dataset_1 = df1.values[difference_locations]
        dataset_2 = df2.values[difference_locations]
        return pd.DataFrame({'rdb': dataset_1, 'rdb_modern': dataset_2},
                            index=changed.index)

In [25]:
df = diff_func(sql_rdb, sql_rdb_mod)

In [22]:
len(df)

36

In [23]:
print(df.to_string())

                                                 rdb                  rdb_modern
ID Column                                                                       
0  HEP_MEDS_RECVD_IND                        Unknown                        None
   CNTRY_USUAL_RESIDENCE               UNITED STATES                        None
   HEP_CONTACT_IND                           Unknown                        None
   HEP_CONTACT_EVER_IND                      Unknown                        None
   STREET_DRUG_INJECTED                      Unknown                        None
   STREET_DRUG_USED                          Unknown                        None
   SEX_PREF                                  Unknown                        None
   INV_CASE_STATUS                        Not a Case                    Probable
   HCV_RNA                                  Positive                        None
   HCV_RNA_DT                             2021-01-03                        None
   PREV_NEG_HEP_TEST_IND    

In [69]:
##Print log file 

## Case 2: New UIDs
### TODO: UIDs that have been introduced by ELR. Not ingested by classic pipeline yet. 